# Preprocess all of Brainiomics Data
**NOTE**: need to set ANTSDIR in Local environment
```
export PATH=/usr/local/antsbin/bin:$PATH
export ANTSPATH=/usr/local/antsbin/bin/
```

In [1]:
%matplotlib inline 
import os
import shutil
import sys
from glob import glob
import pandas as pd
import numpy as np
from nltools.utils import get_resource_path
from nltools.data import Brain_Data, Design_Matrix
from nltools.stats import zscore
import matplotlib
matplotlib.use('Agg')

from nipype.interfaces.io import DataSink, DataGrabber
from nipype.interfaces.utility import Merge, IdentityInterface
from nipype.pipeline.engine import Node, Workflow
from nipype.interfaces.nipy.preprocess import ComputeMask
from nipype.algorithms.rapidart import ArtifactDetect
from nipype.interfaces.ants.segmentation import BrainExtraction, N4BiasFieldCorrection
from nipype.interfaces.ants import Registration, ApplyTransforms
from nipype.interfaces.fsl import MCFLIRT, TOPUP, ApplyTOPUP
from nipype.interfaces.fsl.maths import MeanImage
from nipype.interfaces.fsl import Merge as MERGE
from nipype.interfaces.fsl.utils import Smooth
from nipype.interfaces.nipy.preprocess import Trim


    
netid = 'f00275v'
# base_dir = '/dartfs/rc/lab/P/Psych60/'
# output_interm_dir = os.path.join(base_dir, 'students_output/%s/brainiomics/preprocessing' % netid)
# output_final_dir = os.path.join(base_dir, 'students_output/%s/brainiomics/preprocessed' % netid)
# data_dir = os.path.join(base_dir, 'data/brainomics_data/')
# mask_dir = os.path.join(base_dir, 'resources/masks')

base_dir = '/Volumes/Psych60/'
data_dir = os.path.join(base_dir, 'data/brainomics_data/')
output_interm_dir = os.path.join(base_dir, 'students_output/%s/brainiomics/preprocessing' % netid)

sys.path.append(os.path.join(base_dir, 'resources/preproc'))
from interfaces import (Plot_Coregistration_Montage,
                        Plot_Quality_Control,
                        Plot_Realignment_Parameters,
                        Create_Covariates,
                        Down_Sample_Precision,
                        Squeeze_Nifti)




In [2]:
def brainiomics_preprocessing_workflow(subject_id = None, data_dir=None, output_interm_dir=None, output_final_dir=None, 
                                       mask_dir=None, apply_trim = 0, apply_n4 = True, 
                                       apply_smooth = 6, tr_length = 2.4, ants_threads = 8):

    ###################################
    ### GLOBALS, PATHS ###
    ###################################
    MNItemplate = os.path.join(mask_dir, 'MNI152_T1_2mm_brain.nii.gz')
    MNItemplatehasskull = os.path.join(mask_dir, 'MNI152_T1_2mm.nii.gz')
    bet_ants_template = os.path.join(mask_dir, 'OASIS_template.nii.gz')
    bet_ants_prob_mask = os.path.join(mask_dir, 'OASIS_BrainCerebellumProbabilityMask.nii.gz')
    bet_ants_registration_mask = os.path.join(mask_dir, 'OASIS_BrainCerebellumRegistrationMask.nii.gz')
    bet_ants_extraction_mask = os.path.join(mask_dir, 'OASIS_BrainCerebellumExtractionMask.nii.gz')

    # ###################################
    # ### DATA INPUT ###
    # ###################################
    # #Create a datagrabber that takes a subid as input and creates func and struct dirs
    datasource = Node(DataGrabber(
        infields=['subject_id'],
        outfields = ['func','struct']),
        name = 'datasource')
    datasource.inputs.base_directory = data_dir
    datasource.inputs.subject_id = subject_id
    datasource.inputs.template = '*'
    datasource.inputs.sort_filelist = True
    datasource.inputs.field_template = {'struct': '%s/raw_T1_raw_anat_defaced.nii.gz',
                                        'func': '%s/raw_fMRI_raw_bold.nii.gz'}
    datasource.inputs.template_args = {'struct' :[['subject_id']],
                                       'func': [['subject_id']]}

#     #Then grab all epis using an Identity Interface which is an iterable node
#     func_scans = Node(IdentityInterface(fields=['scan']), name='func_scans')
#     func_scans.inputs.subject_id  = subject_id

    #####################################
    ## SQUEEZE ##
    #####################################
    # Remove extra dimensions from nifti file
    squeeze = Node(Squeeze_Nifti(), name='squeeze')
    
    
    #####################################
    ## TRIM ##
    #####################################
    trim = Node(Trim(), name='trim')
    trim.inputs.begin_index = apply_trim


    ###################################
    ### REALIGN ###
    ###################################
    realign_fsl = Node(MCFLIRT(), name="realign")
    realign_fsl.inputs.cost = 'mutualinfo'
    realign_fsl.inputs.mean_vol = True
    realign_fsl.inputs.output_type = 'NIFTI_GZ'
    realign_fsl.inputs.save_mats = True
    realign_fsl.inputs.save_rms = True
    realign_fsl.inputs.save_plots = True

    ###################################
    ### MEAN EPIs ###
    ###################################
    # For coregistration after realignment
    mean_epi = Node(MeanImage(), name='mean_epi')
    mean_epi.inputs.dimension = 'T'

    # For after normalization is done to plot checks
    mean_norm_epi = Node(MeanImage(), name='mean_norm_epi')
    mean_norm_epi.inputs.dimension = 'T'

    ###################################
    ### MASK, ART, COV CREATION ###
    ###################################
    compute_mask = Node(ComputeMask(), name='compute_mask')
    compute_mask.inputs.m = .05

    art = Node(ArtifactDetect(), name='art')
    art.inputs.use_differences = [True, False]
    art.inputs.use_norm = True
    art.inputs.norm_threshold = 1
    art.inputs.zintensity_threshold = 3
    art.inputs.mask_type = 'file'
    art.inputs.parameter_source = 'FSL'

    make_cov = Node(Create_Covariates(), name='make_cov')

    ################################
    ### N4 BIAS FIELD CORRECTION ###
    ################################
    if apply_n4:
        n4_correction = Node(N4BiasFieldCorrection(), name='n4_correction')
        n4_correction.inputs.copy_header = True
        n4_correction.inputs.save_bias = False
        n4_correction.inputs.num_threads = ants_threads

    ###################################
    ### BRAIN EXTRACTION ###
    ###################################
    brain_extraction_ants = Node(BrainExtraction(), name='brain_extraction')
    brain_extraction_ants.inputs.dimension = 3
    brain_extraction_ants.inputs.use_floatingpoint_precision = 1
    brain_extraction_ants.inputs.num_threads = ants_threads
    brain_extraction_ants.inputs.brain_probability_mask = bet_ants_prob_mask
    brain_extraction_ants.inputs.keep_temporary_files = 1
    brain_extraction_ants.inputs.brain_template = bet_ants_template
    brain_extraction_ants.inputs.extraction_registration_mask = bet_ants_registration_mask
    brain_extraction_ants.inputs.out_prefix = 'bet'

    ###################################
    ### COREGISTRATION ###
    ###################################
    coregistration = Node(Registration(), name='coregistration')
    coregistration.inputs.float = False
    coregistration.inputs.output_transform_prefix = "meanEpi2highres"
    coregistration.inputs.transforms = ['Rigid']
    coregistration.inputs.transform_parameters = [(0.1,), (0.1,)]
    coregistration.inputs.number_of_iterations = [[1000, 500, 250, 100]]
    coregistration.inputs.dimension = 3
    coregistration.inputs.num_threads = ants_threads
    coregistration.inputs.write_composite_transform = True
    coregistration.inputs.collapse_output_transforms = True
    coregistration.inputs.metric = ['MI']
    coregistration.inputs.metric_weight = [1]
    coregistration.inputs.radius_or_number_of_bins = [32]
    coregistration.inputs.sampling_strategy = ['Regular']
    coregistration.inputs.sampling_percentage = [0.25]
    coregistration.inputs.convergence_threshold = [1e-08]
    coregistration.inputs.convergence_window_size = [10]
    coregistration.inputs.smoothing_sigmas = [[3, 2, 1, 0]]
    coregistration.inputs.sigma_units = ['mm']
    coregistration.inputs.shrink_factors = [[4, 3, 2, 1]]
    coregistration.inputs.use_estimate_learning_rate_once = [True]
    coregistration.inputs.use_histogram_matching = [False]
    coregistration.inputs.initial_moving_transform_com = True
    coregistration.inputs.output_warped_image = True
    coregistration.inputs.winsorize_lower_quantile = 0.01
    coregistration.inputs.winsorize_upper_quantile = 0.99

    ###################################
    ### NORMALIZATION ###
    ###################################
    # Settings Explanations
    # Only a few key settings are worth adjusting and most others relate to how ANTs optimizer starts or iterates and won't make a ton of difference
    # Brian Avants referred to these settings as the last "best tested" when he was aligning fMRI data: https://github.com/ANTsX/ANTsRCore/blob/master/R/antsRegistration.R#L275
    # Things that matter the most:
    # smoothing_sigmas:
    # how much gaussian smoothing to apply when performing registration, probably want the upper limit of this to match the resolution that the data is collected at e.g. 3mm
    # Old settings [[3,2,1,0]]*3
    # shrink_factors
    # The coarseness with which to do registration
    # Old settings [[8,4,2,1]] * 3
    # >= 8 may result is some problems causing big chunks of cortex with little fine grain spatial structure to be moved to other parts of cortex
    # Other settings
    # transform_parameters:
    # how much regularization to do for fitting that transformation
    # for syn this pertains to both the gradient regularization term, and the flow, and elastic terms. Leave the syn settings alone as they seem to be the most well tested across published data sets
    # radius_or_number_of_bins
    # This is the bin size for MI metrics and 32 is probably adequate for most use cases. Increasing this might increase precision (e.g. to 64) but takes exponentially longer
    # use_histogram_matching
    # Use image intensity distribution to guide registration
    # Leave it on for within modality registration (e.g. T1 -> MNI), but off for between modality registration (e.g. EPI -> T1)
    # convergence_threshold
    # threshold for optimizer
    # convergence_window_size
    # how many samples should optimizer average to compute threshold?
    # sampling_strategy
    # what strategy should ANTs use to initialize the transform. Regular here refers to approximately random sampling around the center of the image mass

    normalization = Node(Registration(), name='normalization')
    normalization.inputs.float = False
    normalization.inputs.collapse_output_transforms = True
    normalization.inputs.convergence_threshold = [1e-06]
    normalization.inputs.convergence_window_size = [10]
    normalization.inputs.dimension = 3
    normalization.inputs.fixed_image = MNItemplate
    normalization.inputs.initial_moving_transform_com = True
    normalization.inputs.metric = ['MI', 'MI', 'CC']
    normalization.inputs.metric_weight = [1.0]*3
    normalization.inputs.number_of_iterations = [[1000, 500, 250, 100],
                                                 [1000, 500, 250, 100],
                                                 [100, 70, 50, 20]]
    normalization.inputs.num_threads = ants_threads
    normalization.inputs.output_transform_prefix = 'anat2template'
    normalization.inputs.output_inverse_warped_image = True
    normalization.inputs.output_warped_image = True
    normalization.inputs.radius_or_number_of_bins = [32, 32, 4]
    normalization.inputs.sampling_percentage = [0.25, 0.25, 1]
    normalization.inputs.sampling_strategy = ['Regular', 'Regular', 'None']
    normalization.inputs.shrink_factors = [[8, 4, 2, 1]]*3
    normalization.inputs.sigma_units = ['vox']*3
    normalization.inputs.smoothing_sigmas = [[3, 2, 1, 0]]*3
    normalization.inputs.transforms = ['Rigid', 'Affine', 'SyN']
    normalization.inputs.transform_parameters = [(0.1,), (0.1,), (0.1, 3.0, 0.0)]
    normalization.inputs.use_histogram_matching = True
    normalization.inputs.winsorize_lower_quantile = 0.005
    normalization.inputs.winsorize_upper_quantile = 0.995
    normalization.inputs.write_composite_transform = True


    ###################################
    ### APPLY TRANSFORMS AND SMOOTH ###
    ###################################
    merge_transforms = Node(Merge(2), iterfield=['in2'], name='merge_transforms')

    # Used for epi -> mni, via (coreg + norm)
    apply_transforms = Node(ApplyTransforms(), iterfield=['input_image'], name='apply_transforms')
    apply_transforms.inputs.input_image_type = 3
    apply_transforms.inputs.float = False
    apply_transforms.inputs.num_threads = 12
    apply_transforms.inputs.environ = {}
    apply_transforms.inputs.interpolation = 'BSpline'
    apply_transforms.inputs.invert_transform_flags = [False, False]
    apply_transforms.inputs.reference_image = MNItemplate

    # Used for t1 segmented -> mni, via (norm)
    apply_transform_seg = Node(ApplyTransforms(), name='apply_transform_seg')
    apply_transform_seg.inputs.input_image_type = 3
    apply_transform_seg.inputs.float = False
    apply_transform_seg.inputs.num_threads = 12
    apply_transform_seg.inputs.environ = {}
    apply_transform_seg.inputs.interpolation = 'MultiLabel'
    apply_transform_seg.inputs.invert_transform_flags = [False]
    apply_transform_seg.inputs.reference_image = MNItemplate

    ###################################
    ### PLOTS ###
    ###################################
    plot_realign = Node(Plot_Realignment_Parameters(), name="plot_realign")
    plot_qa = Node(Plot_Quality_Control(), name="plot_qa")
    plot_normalization_check = Node(Plot_Coregistration_Montage(), name="plot_normalization_check")
    plot_normalization_check.inputs.canonical_img = MNItemplatehasskull

    ############################################
    ### FILTER, SMOOTH, DOWNSAMPLE PRECISION ###
    ############################################
    # Use cosanlab_preproc for down sampling
    down_samp = Node(Down_Sample_Precision(), name="down_samp")

    # Use FSL for smoothing
    smooth = Node(Smooth(), name='smooth')
    if isinstance(apply_smooth, list):
        smooth.iterables = ("fwhm", apply_smooth)
    elif isinstance(apply_smooth, int) or isinstance(apply_smooth, float):
        smooth.inputs.fwhm = apply_smooth
    else:
        raise ValueError("apply_smooth must be a list or int/float")

    ###################
    ### OUTPUT NODE ###
    ###################
    # Collect all final outputs in the output dir and get rid of file name additions
    datasink = Node(DataSink(), name='datasink')

    datasink.inputs.base_directory = output_final_dir
    datasink.inputs.container = subject_id

    ############## - NEED TO FIX
    # # Remove substitutions
    # data_dir_parts = data_dir.split('/')[1:]
    # prefix = ['_scan_'] + data_dir_parts + [subject_id] + ['func']
    # func_scan_names = [os.path.split(elem)[-1] for elem in funcs]
    # to_replace = []
    # for elem in func_scan_names:
    #     bold_name = elem.split(subject_id + '_')[-1]
    #     bold_name = bold_name.split('.nii.gz')[0]
    #     to_replace.append(('..'.join(prefix + [elem]), bold_name))
    # datasink.inputs.substitutions = to_replace

    #####################
    ### INIT WORKFLOW ###
    #####################
    # If we have sessions provide the full path to the subject's intermediate directory
    # and only rely on workflow init to create the session container *within* that directory
    # Otherwise just point to the intermediate directory and let the workflow init create the subject container within the intermediate directory

    workflow = Workflow(name=subject_id)
    workflow.base_dir = output_interm_dir

    ############################
    ######### PART (1a) #########
    # func -> discorr -> trim -> realign
    # OR
    # func -> trim -> realign
    # OR
    # func -> discorr -> realign
    # OR
    # func -> realign
    ############################


    workflow.connect([
        (datasource, squeeze, [('func', 'in_file')]),
        (squeeze, trim, [('out_file', 'in_file')]),
        (trim, realign_fsl, [('out_file', 'in_file')])
    ])

    ############################
    ######### PART (1n) #########
    # anat -> N4 -> bet
    # OR
    # anat -> bet
    ############################
    #     n4_correction.inputs.input_image = anat

    if apply_n4:
        workflow.connect([
            (datasource, n4_correction, [('struct', 'input_image')]),
            (n4_correction, brain_extraction_ants, [('output_image', 'anatomical_image')])
        ])
    else:
        workflow.connect([
            (datasource, brain_extraction_ants, [('struct', 'anatomical_image')])
        ])

    ##########################################
    ############### PART (2) #################
    # realign -> coreg -> mni (via t1)
    # t1 -> mni
    # covariate creation
    # plot creation
    ###########################################

    workflow.connect([
        (realign_fsl, plot_realign, [('par_file', 'realignment_parameters')]),
        (realign_fsl, plot_qa, [('out_file', 'dat_img')]),
        (realign_fsl, art, [('out_file', 'realigned_files'),
                            ('par_file', 'realignment_parameters')]),
        (realign_fsl, mean_epi, [('out_file', 'in_file')]),
        (realign_fsl, make_cov, [('par_file', 'realignment_parameters')]),
        (mean_epi, compute_mask, [('out_file', 'mean_volume')]),
        (compute_mask, art, [('brain_mask', 'mask_file')]),
        (art, make_cov, [('outlier_files', 'spike_id')]),
        (art, plot_realign, [('outlier_files', 'outliers')]),
        (plot_qa, make_cov, [('fd_outliers', 'fd_outliers')]),
        (brain_extraction_ants, coregistration, [('BrainExtractionBrain', 'fixed_image')]),
        (mean_epi, coregistration, [('out_file', 'moving_image')]),
        (brain_extraction_ants, normalization, [('BrainExtractionBrain', 'moving_image')]),
        (coregistration, merge_transforms, [('composite_transform', 'in2')]),
        (normalization, merge_transforms, [('composite_transform', 'in1')]),
        (merge_transforms, apply_transforms, [('out', 'transforms')]),
        (realign_fsl, apply_transforms, [('out_file', 'input_image')]),
        (apply_transforms, mean_norm_epi, [('output_image', 'in_file')]),
        (normalization, apply_transform_seg, [('composite_transform', 'transforms')]),
        (brain_extraction_ants, apply_transform_seg, [('BrainExtractionSegmentation', 'input_image')]),
        (mean_norm_epi, plot_normalization_check, [('out_file', 'wra_img')])
    ])

    ##################################################
    ################### PART (3) #####################
    # epi (in mni) -> filter -> smooth -> down sample
    # OR
    # epi (in mni) -> filter -> down sample
    # OR
    # epi (in mni) -> smooth -> down sample
    # OR
    # epi (in mni) -> down sample
    ###################################################


    workflow.connect([
        (apply_transforms, smooth, [('output_image', 'in_file')]),
        (smooth, down_samp, [('smoothed_file', 'in_file')])
        ])


    ##########################################
    ############### PART (4) #################
    # down sample -> save
    # plots -> save
    # covs -> save
    # t1 (in mni) -> save
    # t1 segmented masks (in mni) -> save
    # realignment parms -> save
    ##########################################

    workflow.connect([
        (down_samp, datasink, [('out_file', 'functional.@down_samp')]),
        (plot_realign, datasink, [('plot', 'functional.@plot_realign')]),
        (plot_qa, datasink, [('plot', 'functional.@plot_qa')]),
        (plot_normalization_check, datasink, [('plot', 'functional.@plot_normalization')]),
        (make_cov, datasink, [('covariates', 'functional.@covariates')]),
        (normalization, datasink, [('warped_image', 'structural.@normanat')]),
        (apply_transform_seg, datasink, [('output_image', 'structural.@normanatseg')]),
        (realign_fsl, datasink, [('par_file', 'functional.@motionparams')])
    ])

    if not os.path.exists(os.path.join(output_final_dir, 'pipeline.png')):
        workflow.write_graph(dotfilename=os.path.join(output_final_dir, 'pipeline'), format='png')

    print(f"Creating workflow for subject: {subject_id}")
    if ants_threads != 8:
        print(f"ANTs will utilize the user-requested {ants_threads} threads for parallel processing.")
    return workflow

In [3]:
trim = 0
smooth = 6
subject_id = 'S01'
tr = 2.4
ants_threads = 6

wf = brainiomics_preprocessing_workflow(subject_id=subject_id, 
                                        data_dir=data_dir,
                                        output_interm_dir=output_interm_dir, 
                                        output_final_dir=output_final_dir, 
                                        mask_dir=mask_dir, 
                                        apply_n4=True, 
                                        apply_smooth=smooth, 
                                        tr_length=tr, 
                                        apply_trim=trim, 
                                        ants_threads=ants_threads)
wf.run()
# wf.run('MultiProc', plugin_args={'n_procs': 1})

Creating workflow for subject: S01
ANTs will utilize the user-requested 6 threads for parallel processing.
190421-18:10:45,690 nipype.workflow INFO:
	 Workflow S01 settings: ['check', 'execution', 'logging', 'monitoring']
190421-18:10:45,879 nipype.workflow INFO:
	 Running serially.
190421-18:10:45,881 nipype.workflow INFO:
	 [Node] Setting-up "S01.datasource" in "/dartfs/rc/lab/P/Psych60/students_output/f00275v/brainiomics/preprocessing/S01/datasource".
190421-18:10:45,967 nipype.workflow INFO:
	 [Node] Running "datasource" ("nipype.interfaces.io.DataGrabber")
190421-18:10:46,13 nipype.workflow INFO:
	 [Node] Finished "S01.datasource".
190421-18:10:46,15 nipype.workflow INFO:
	 [Node] Setting-up "S01.n4_correction" in "/dartfs/rc/lab/P/Psych60/students_output/f00275v/brainiomics/preprocessing/S01/n4_correction".
190421-18:10:46,30 nipype.workflow INFO:
	 [Node] Cached "S01.n4_correction" - collecting precomputed outputs
190421-18:10:46,31 nipype.workflow INFO:
	 [Node] "S01.n4_correct

/optnfs/el7/jupyterhub/envs/Psych60-Chang/lib/python3.7/site-packages/scipy/ndimage/measurements.py:272: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  return _nd_image.find_objects(input, max_label)


190421-18:14:54,433 nipype.workflow INFO:
	 [Node] Finished "S01.plot_qa".
190421-18:14:54,435 nipype.workflow INFO:
	 [Node] Setting-up "S01.make_cov" in "/dartfs/rc/lab/P/Psych60/students_output/f00275v/brainiomics/preprocessing/S01/make_cov".
190421-18:14:54,529 nipype.workflow INFO:
	 [Node] Running "make_cov" ("interfaces.Create_Covariates")
190421-18:14:54,566 nipype.workflow WARNING:
	 [Node] Error on "S01.make_cov" (/dartfs/rc/lab/P/Psych60/students_output/f00275v/brainiomics/preprocessing/S01/make_cov)
190421-18:14:54,587 nipype.workflow ERROR:
	 Node make_cov failed to run on host nikola.dartmouth.edu.
190421-18:14:54,591 nipype.workflow ERROR:
	 Saving crash info to /dartfs/rc/lab/P/Psych60/labs/S2019/crash-20190421-181454-f00275v-make_cov-c10ad8f4-89de-42c7-a5b9-3b62b6549dab.pklz
Traceback (most recent call last):
  File "/optnfs/el7/jupyterhub/envs/Psych60-Chang/lib/python3.7/site-packages/nipype/pipeline/plugins/linear.py", line 48, in run
    node.run(updatehash=updateha

/dartfs/rc/lab/P/Psych60/resources/preproc/interfaces.py:465: FutureWarning: read_table is deprecated, use read_csv instead.
  sep=r"\s*", names=['ra' + str(x) for x in range(1, 7)])
/dartfs/rc/lab/P/Psych60/resources/preproc/interfaces.py:465: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  sep=r"\s*", names=['ra' + str(x) for x in range(1, 7)])


RuntimeError: Workflow did not execute cleanly. Check log for details

In [4]:
!nipypecli crash /dartfs/rc/lab/P/Psych60/labs/S2019/crash-20190421-181454-f00275v-make_cov-c10ad8f4-89de-42c7-a5b9-3b62b6549dab.pklz




Traceback (most recent call last):
  File "/optnfs/el7/jupyterhub/envs/Psych60-Chang-3.6.8/bin/nipypecli", line 10, in <module>
    sys.exit(cli())
  File "/optnfs/el7/jupyterhub/envs/Psych60-Chang-3.6.8/lib/python3.6/site-packages/click/core.py", line 764, in __call__
    return self.main(*args, **kwargs)
  File "/optnfs/el7/jupyterhub/envs/Psych60-Chang-3.6.8/lib/python3.6/site-packages/click/core.py", line 717, in main
    rv = self.invoke(ctx)
  File "/optnfs/el7/jupyterhub/envs/Psych60-Chang-3.6.8/lib/python3.6/site-packages/click/core.py", line 1137, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "/optnfs/el7/jupyterhub/envs/Psych60-Chang-3.6.8/lib/python3.6/site-packages/click/core.py", line 956, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/optnfs/el7/jupyterhub/envs/Psych60-Chang-3.6.8/lib/python3.6/site-packages/click/core.py", line 555, in invoke
    return callback(*args, **kwargs)
  File "/optnfs/el7/jupyterhub/envs/Psy

# Only Run Realignment on Raw Data

In [8]:
def brainiomics_realign_workflow(subject_id = None, data_dir=None, output_interm_dir=None, output_final_dir=None, 
                                       mask_dir=None, apply_trim = 0, tr_length = 2.4):

    ###################################
    ### GLOBALS, PATHS ###
    ###################################
    MNItemplate = os.path.join(mask_dir, 'MNI152_T1_2mm_brain.nii.gz')
    MNItemplatehasskull = os.path.join(mask_dir, 'MNI152_T1_2mm.nii.gz')

    # ###################################
    # ### DATA INPUT ###
    # ###################################
    # #Create a datagrabber that takes a subid as input and creates func and struct dirs
    datasource = Node(DataGrabber(
        infields=['subject_id'],
        outfields = ['func','struct']),
        name = 'datasource')
    datasource.inputs.base_directory = data_dir
    datasource.inputs.subject_id = subject_id
    datasource.inputs.template = '*'
    datasource.inputs.sort_filelist = True
    datasource.inputs.field_template = {'struct': '%s/raw_T1_raw_anat_defaced.nii.gz',
                                        'func': '%s/raw_fMRI_raw_bold.nii.gz'}
    datasource.inputs.template_args = {'struct' :[['subject_id']],
                                       'func': [['subject_id']]}


    #####################################
    ## SQUEEZE ##
    #####################################
    # Remove extra dimensions from nifti file
    squeeze = Node(Squeeze_Nifti(), name='squeeze')
    
    
    #####################################
    ## TRIM ##
    #####################################
    trim = Node(Trim(), name='trim')
    trim.inputs.begin_index = apply_trim


    ###################################
    ### REALIGN ###
    ###################################
    realign_fsl = Node(MCFLIRT(), name="realign")
    realign_fsl.inputs.cost = 'mutualinfo'
    realign_fsl.inputs.mean_vol = True
    realign_fsl.inputs.output_type = 'NIFTI_GZ'
    realign_fsl.inputs.save_mats = True
    realign_fsl.inputs.save_rms = True
    realign_fsl.inputs.save_plots = True

    ###################################
    ### MEAN EPIs ###
    ###################################
    # For coregistration after realignment
    mean_epi = Node(MeanImage(), name='mean_epi')
    mean_epi.inputs.dimension = 'T'

    # For after normalization is done to plot checks
    mean_norm_epi = Node(MeanImage(), name='mean_norm_epi')
    mean_norm_epi.inputs.dimension = 'T'

    ###################################
    ### MASK, ART, COV CREATION ###
    ###################################
    compute_mask = Node(ComputeMask(), name='compute_mask')
    compute_mask.inputs.m = .05

    art = Node(ArtifactDetect(), name='art')
    art.inputs.use_differences = [True, False]
    art.inputs.use_norm = True
    art.inputs.norm_threshold = 1
    art.inputs.zintensity_threshold = 3
    art.inputs.mask_type = 'file'
    art.inputs.parameter_source = 'FSL'

    ###################################
    ### PLOTS ###
    ###################################
    plot_realign = Node(Plot_Realignment_Parameters(), name="plot_realign")

    #####################
    ### INIT WORKFLOW ###
    #####################
    # If we have sessions provide the full path to the subject's intermediate directory
    # and only rely on workflow init to create the session container *within* that directory
    # Otherwise just point to the intermediate directory and let the workflow init create the subject container within the intermediate directory

    workflow = Workflow(name=subject_id)
    workflow.base_dir = output_interm_dir

    ############################
    ######### PART (1a) #########
    # func -> discorr -> trim -> realign
    # OR
    # func -> trim -> realign
    # OR
    # func -> discorr -> realign
    # OR
    # func -> realign
    ############################


    workflow.connect([
        (datasource, squeeze, [('func', 'in_file')]),
        (squeeze, trim, [('out_file', 'in_file')]),
        (trim, realign_fsl, [('out_file', 'in_file')])
    ])

    ##########################################
    ############### PART (2) #################
    # realign -> coreg -> mni (via t1)
    # t1 -> mni
    # covariate creation
    # plot creation
    ###########################################

    workflow.connect([
        (realign_fsl, plot_realign, [('par_file', 'realignment_parameters')]),
        (realign_fsl, art, [('out_file', 'realigned_files'),
                            ('par_file', 'realignment_parameters')]),
        (realign_fsl, mean_epi, [('out_file', 'in_file')]),
        (mean_epi, compute_mask, [('out_file', 'mean_volume')]),
        (compute_mask, art, [('brain_mask', 'mask_file')]),
        (art, plot_realign, [('outlier_files', 'outliers')]),
    ])
    
    workflow.write_graph(dotfilename=os.path.join(output_interm_dir, 'pipeline'), format='png')
    return workflow

In [25]:
subject_id = 'S01'
wf = brainiomics_realign_workflow(subject_id=subject_id, 
                                        data_dir=data_dir,
                                        output_interm_dir=output_interm_dir, 
                                        output_final_dir=output_final_dir, 
                                        mask_dir=mask_dir, 
                                        tr_length=tr, 
                                        apply_trim=trim)
wf.run()

190421-22:02:15,678 nipype.workflow INFO:
	 Generated workflow graph: /dartfs/rc/lab/P/Psych60/students_output/f00275v/brainiomics/preprocessing/pipeline.png (graph2use=hierarchical, simple_form=True).
190421-22:02:15,684 nipype.workflow INFO:
	 Workflow S01 settings: ['check', 'execution', 'logging', 'monitoring']
190421-22:02:15,843 nipype.workflow INFO:
	 Running serially.
190421-22:02:15,844 nipype.workflow INFO:
	 [Node] Setting-up "S01.datasource" in "/dartfs/rc/lab/P/Psych60/students_output/f00275v/brainiomics/preprocessing/S01/datasource".
190421-22:02:15,896 nipype.workflow INFO:
	 [Node] Running "datasource" ("nipype.interfaces.io.DataGrabber")
190421-22:02:15,958 nipype.workflow INFO:
	 [Node] Finished "S01.datasource".
190421-22:02:15,959 nipype.workflow INFO:
	 [Node] Setting-up "S01.squeeze" in "/dartfs/rc/lab/P/Psych60/students_output/f00275v/brainiomics/preprocessing/S01/squeeze".
190421-22:02:16,5 nipype.workflow INFO:
	 [Node] Running "squeeze" ("interfaces.Squeeze_Ni

In [23]:
trim = 0
subject_id = 'S01'
tr = 2.4

sub_list = [os.path.basename(x) for x in glob(os.path.join(data_dir, 'S*'))]
completed = [os.path.basename(x) for x in glob(os.path.join(output_interm_dir,'S*'))]

for subject_id in [x for x in sub_list if x not in completed]:
    wf = brainiomics_realign_workflow(subject_id=subject_id, 
                                            data_dir=data_dir,
                                            output_interm_dir=output_interm_dir, 
                                            output_final_dir=output_final_dir, 
                                            mask_dir=mask_dir, 
                                            tr_length=tr, 
                                            apply_trim=trim)
    wf.run()

190421-19:56:39,95 nipype.workflow INFO:
	 Generated workflow graph: /dartfs/rc/lab/P/Psych60/students_output/f00275v/brainiomics/preprocessing/pipeline.png (graph2use=hierarchical, simple_form=True).
190421-19:56:39,100 nipype.workflow INFO:
	 Workflow S17 settings: ['check', 'execution', 'logging', 'monitoring']
190421-19:56:39,270 nipype.workflow INFO:
	 Running serially.
190421-19:56:39,272 nipype.workflow INFO:
	 [Node] Setting-up "S17.datasource" in "/dartfs/rc/lab/P/Psych60/students_output/f00275v/brainiomics/preprocessing/S17/datasource".
190421-19:56:39,327 nipype.workflow INFO:
	 [Node] Running "datasource" ("nipype.interfaces.io.DataGrabber")
190421-19:56:39,394 nipype.workflow INFO:
	 [Node] Finished "S17.datasource".
190421-19:56:39,396 nipype.workflow INFO:
	 [Node] Setting-up "S17.squeeze" in "/dartfs/rc/lab/P/Psych60/students_output/f00275v/brainiomics/preprocessing/S17/squeeze".
190421-19:56:39,460 nipype.workflow INFO:
	 [Node] Running "squeeze" ("interfaces.Squeeze_N

190421-20:04:46,470 nipype.workflow INFO:
	 [Node] Setting-up "S14.datasource" in "/dartfs/rc/lab/P/Psych60/students_output/f00275v/brainiomics/preprocessing/S14/datasource".
190421-20:04:46,521 nipype.workflow INFO:
	 [Node] Running "datasource" ("nipype.interfaces.io.DataGrabber")
190421-20:04:46,569 nipype.workflow INFO:
	 [Node] Finished "S14.datasource".
190421-20:04:46,571 nipype.workflow INFO:
	 [Node] Setting-up "S14.squeeze" in "/dartfs/rc/lab/P/Psych60/students_output/f00275v/brainiomics/preprocessing/S14/squeeze".
190421-20:04:46,619 nipype.workflow INFO:
	 [Node] Running "squeeze" ("interfaces.Squeeze_Nifti")
190421-20:05:17,154 nipype.workflow INFO:
	 [Node] Finished "S14.squeeze".
190421-20:05:17,158 nipype.workflow INFO:
	 [Node] Setting-up "S14.trim" in "/dartfs/rc/lab/P/Psych60/students_output/f00275v/brainiomics/preprocessing/S14/trim".
190421-20:05:17,268 nipype.workflow INFO:
	 [Node] Running "trim" ("nipype.interfaces.nipy.preprocess.Trim")
190421-20:05:48,10 nipyp

190421-20:12:38,234 nipype.workflow INFO:
	 [Node] Running "squeeze" ("interfaces.Squeeze_Nifti")
190421-20:13:06,84 nipype.workflow INFO:
	 [Node] Finished "S20.squeeze".
190421-20:13:06,87 nipype.workflow INFO:
	 [Node] Setting-up "S20.trim" in "/dartfs/rc/lab/P/Psych60/students_output/f00275v/brainiomics/preprocessing/S20/trim".
190421-20:13:06,156 nipype.workflow INFO:
	 [Node] Running "trim" ("nipype.interfaces.nipy.preprocess.Trim")
190421-20:13:35,549 nipype.workflow INFO:
	 [Node] Finished "S20.trim".
190421-20:13:35,553 nipype.workflow INFO:
	 [Node] Setting-up "S20.realign" in "/dartfs/rc/lab/P/Psych60/students_output/f00275v/brainiomics/preprocessing/S20/realign".
190421-20:13:35,672 nipype.workflow INFO:
	 [Node] Running "realign" ("nipype.interfaces.fsl.preprocess.MCFLIRT"), a CommandLine Interface with command:
mcflirt -in /dartfs/rc/lab/P/Psych60/students_output/f00275v/brainiomics/preprocessing/S20/trim/raw_fMRI_raw_bold_fixed_trim.nii.gz -cost mutualinfo -meanvol -out 

190421-20:21:22,717 nipype.workflow INFO:
	 [Node] Finished "S12.trim".
190421-20:21:22,720 nipype.workflow INFO:
	 [Node] Setting-up "S12.realign" in "/dartfs/rc/lab/P/Psych60/students_output/f00275v/brainiomics/preprocessing/S12/realign".
190421-20:21:22,820 nipype.workflow INFO:
	 [Node] Running "realign" ("nipype.interfaces.fsl.preprocess.MCFLIRT"), a CommandLine Interface with command:
mcflirt -in /dartfs/rc/lab/P/Psych60/students_output/f00275v/brainiomics/preprocessing/S12/trim/raw_fMRI_raw_bold_fixed_trim.nii.gz -cost mutualinfo -meanvol -out /dartfs/rc/lab/P/Psych60/students_output/f00275v/brainiomics/preprocessing/S12/realign/raw_fMRI_raw_bold_fixed_trim_mcf.nii.gz -mats -plots -rmsabs -rmsrel
190421-20:24:25,776 nipype.workflow INFO:
	 [Node] Finished "S12.realign".
190421-20:24:25,778 nipype.workflow INFO:
	 [Node] Setting-up "S12.mean_epi" in "/dartfs/rc/lab/P/Psych60/students_output/f00275v/brainiomics/preprocessing/S12/mean_epi".
190421-20:24:25,910 nipype.workflow INFO:

190421-20:34:01,876 nipype.workflow INFO:
	 [Node] Finished "S29.realign".
190421-20:34:01,879 nipype.workflow INFO:
	 [Node] Setting-up "S29.mean_epi" in "/dartfs/rc/lab/P/Psych60/students_output/f00275v/brainiomics/preprocessing/S29/mean_epi".
190421-20:34:01,939 nipype.workflow INFO:
	 [Node] Running "mean_epi" ("nipype.interfaces.fsl.maths.MeanImage"), a CommandLine Interface with command:
fslmaths /dartfs/rc/lab/P/Psych60/students_output/f00275v/brainiomics/preprocessing/S29/realign/raw_fMRI_raw_bold_fixed_trim_mcf.nii.gz -Tmean /dartfs/rc/lab/P/Psych60/students_output/f00275v/brainiomics/preprocessing/S29/mean_epi/raw_fMRI_raw_bold_fixed_trim_mcf_mean.nii.gz
190421-20:34:05,463 nipype.workflow INFO:
	 [Node] Finished "S29.mean_epi".
190421-20:34:05,465 nipype.workflow INFO:
	 [Node] Setting-up "S29.compute_mask" in "/dartfs/rc/lab/P/Psych60/students_output/f00275v/brainiomics/preprocessing/S29/compute_mask".
190421-20:34:05,510 nipype.workflow INFO:
	 [Node] Running "compute_mask

...

190421-20:52:36,426 nipype.workflow INFO:
	 [Node] Finished "S22.plot_realign".
190421-20:52:36,907 nipype.workflow INFO:
	 Generated workflow graph: /dartfs/rc/lab/P/Psych60/students_output/f00275v/brainiomics/preprocessing/pipeline.png (graph2use=hierarchical, simple_form=True).
190421-20:52:36,912 nipype.workflow INFO:
	 Workflow S28 settings: ['check', 'execution', 'logging', 'monitoring']
190421-20:52:37,43 nipype.workflow INFO:
	 Running serially.
190421-20:52:37,45 nipype.workflow INFO:
	 [Node] Setting-up "S28.datasource" in "/dartfs/rc/lab/P/Psych60/students_output/f00275v/brainiomics/preprocessing/S28/datasource".
190421-20:52:37,101 nipype.workflow INFO:
	 [Node] Running "datasource" ("nipype.interfaces.io.DataGrabber")
190421-20:52:37,156 nipype.workflow INFO:
	 [Node] Finished "S28.datasource".
190421-20:52:37,158 nipype.workflow INFO:
	 [Node] Setting-up "S28.squeeze" in "/dartfs/rc/lab/P/Psych60/students_output/f00275v/brainiomics/preprocessing/S28/squeeze".
190421-20:52

/dartfs/rc/lab/P/Psych60/resources/preproc/interfaces.py:279: UserWarning: loadtxt: Empty input file: "/dartfs/rc/lab/P/Psych60/students_output/f00275v/brainiomics/preprocessing/S18/art/art.raw_fMRI_raw_bold_fixed_trim_mcf_outliers.txt"
  outliers = np.loadtxt(self.inputs.outliers)


190421-20:59:45,808 nipype.workflow INFO:
	 [Node] Finished "S18.plot_realign".
190421-20:59:46,250 nipype.workflow INFO:
	 Generated workflow graph: /dartfs/rc/lab/P/Psych60/students_output/f00275v/brainiomics/preprocessing/pipeline.png (graph2use=hierarchical, simple_form=True).
190421-20:59:46,256 nipype.workflow INFO:
	 Workflow S04 settings: ['check', 'execution', 'logging', 'monitoring']
190421-20:59:46,391 nipype.workflow INFO:
	 Running serially.
190421-20:59:46,393 nipype.workflow INFO:
	 [Node] Setting-up "S04.datasource" in "/dartfs/rc/lab/P/Psych60/students_output/f00275v/brainiomics/preprocessing/S04/datasource".
190421-20:59:46,446 nipype.workflow INFO:
	 [Node] Running "datasource" ("nipype.interfaces.io.DataGrabber")
190421-20:59:46,493 nipype.workflow INFO:
	 [Node] Finished "S04.datasource".
190421-20:59:46,495 nipype.workflow INFO:
	 [Node] Setting-up "S04.squeeze" in "/dartfs/rc/lab/P/Psych60/students_output/f00275v/brainiomics/preprocessing/S04/squeeze".
190421-20:

/dartfs/rc/lab/P/Psych60/resources/preproc/interfaces.py:279: UserWarning: loadtxt: Empty input file: "/dartfs/rc/lab/P/Psych60/students_output/f00275v/brainiomics/preprocessing/S04/art/art.raw_fMRI_raw_bold_fixed_trim_mcf_outliers.txt"
  outliers = np.loadtxt(self.inputs.outliers)


190421-21:03:31,909 nipype.workflow INFO:
	 [Node] Finished "S04.plot_realign".
190421-21:03:32,334 nipype.workflow INFO:
	 Generated workflow graph: /dartfs/rc/lab/P/Psych60/students_output/f00275v/brainiomics/preprocessing/pipeline.png (graph2use=hierarchical, simple_form=True).
190421-21:03:32,339 nipype.workflow INFO:
	 Workflow S21 settings: ['check', 'execution', 'logging', 'monitoring']
190421-21:03:32,484 nipype.workflow INFO:
	 Running serially.
190421-21:03:32,486 nipype.workflow INFO:
	 [Node] Setting-up "S21.datasource" in "/dartfs/rc/lab/P/Psych60/students_output/f00275v/brainiomics/preprocessing/S21/datasource".
190421-21:03:32,561 nipype.workflow INFO:
	 [Node] Running "datasource" ("nipype.interfaces.io.DataGrabber")
190421-21:03:32,610 nipype.workflow INFO:
	 [Node] Finished "S21.datasource".
190421-21:03:32,611 nipype.workflow INFO:
	 [Node] Setting-up "S21.squeeze" in "/dartfs/rc/lab/P/Psych60/students_output/f00275v/brainiomics/preprocessing/S21/squeeze".
190421-21:

In [15]:
!nipypecli crash /dartfs/rc/lab/P/Psych60/labs/S2019/crash-20190421-192235-f00275v-datasource-0ceee35e-4c5c-4757-8f4d-c4e4ef62b0ef.pklz





File: /dartfs/rc/lab/P/Psych60/labs/S2019/crash-20190421-192235-f00275v-datasource-0ceee35e-4c5c-4757-8f4d-c4e4ef62b0ef.pklz
Node: S30.datasource
Working directory: /dartfs/rc/lab/P/Psych60/students_output/f00275v/brainiomics/preprocessing/S30/datasource


Node inputs:

__instance_traits__ = {'subject_id': <traits.traits.CTrait object at 0x2aaf58bd9978>, 'field_template_items': <traits.traits.CTrait object at 0x2aaf58bd9cc0>, 'field_template': <traits.traits.CTrait object at 0x2aaf58be8390>}
base_directory = /dartfs/rc/lab/P/Psych60/data/brainomics_data/
drop_blank_outputs = False
field_template = {'struct': '%s/raw_T1_raw_anat_defaced.nii.gz', 'func': '%s/raw_fMRI_raw_bold.nii.gz'}
raise_on_empty = True
sort_filelist = True
subject_id = S30
template = *
template_args = {'struct': [['subject_id']], 'func': [['subject_id']]}



Traceback: 
Traceback (most recent call last):
  File "/optnfs/el7/jupyterhub/envs/Psych60-Chang/lib/python3.7/site-packages/nipype/pipeline/plugins/linear.py"

# Move Realignment Parameters to Data Folder

In [41]:
file_list = glob(os.path.join(output_interm_dir,'S*/*realign/raw*par'))

for f in file_list:
    sub = os.path.dirname(f).split('/')[-2]
    shutil.copy(f, os.path.join(data_dir, sub, 'realignment_parameters.txt' ))

# Run First Level Model With Denoising on all subjects

In [44]:
tr = 2.4
fwhm = 6

def make_motion_covariates(mc):
    z_mc = zscore(mc)
    all_mc = pd.concat([z_mc, z_mc**2, z_mc.diff(), z_mc.diff()**2], axis=1)
    all_mc.fillna(value=0, inplace=True)
    return Design_Matrix(all_mc, sampling_freq=1/tr)

sub = 'S01'
sub_list = [os.path.basename(x) for x in glob(os.path.join(data_dir, 'S*'))]
sub_list = [x for x in sub_list if x != 'S30']
completed = [os.path.dirname(x).split('/')[-1] for x in glob(os.path.join(data_dir, '*', 'denoised_smoothed_preprocessed_fMRI_bold.nii.gz'))]
for sub in [x for x in sub_list if x not in completed]:
    print(sub)
    file_name = os.path.join(data_dir, sub ,'preprocessed_fMRI_bold.nii.gz')
    data = Brain_Data(file_name)
    n_tr = len(data)
    spikes = data.find_spikes(global_spike_cutoff=3, diff_spike_cutoff=3)
    mc = pd.read_csv(os.path.join(data_dir, sub ,'realignment_parameters.txt'), sep='\s', header=None)
    mc_cov = make_motion_covariates(mc)
    df = pd.read_csv(os.path.join(data_dir, 'Design_Matrix.csv'))
    dm = Design_Matrix(df, sampling_freq=1/tr)
    dm = dm.loc[:n_tr-1,:]
    dm_conv = dm.convolve()
    dm_conv_filt = dm_conv.add_dct_basis(duration=128)
    dm_conv_filt_poly = dm_conv_filt.add_poly(order=2, include_lower=True)
    dm_conv_filt_poly_cov = dm_conv_filt_poly.append(mc_cov, axis=1).append(Design_Matrix(spikes.iloc[:,1:], sampling_freq=1/tr), axis=1)
    data.X = dm_conv_filt_poly_cov
    stats = data.regress()
    smoothed = stats['beta'].smooth(fwhm=fwhm)
    smoothed.write(os.path.join(data_dir, sub, 'betas_denoised_smoothed_preprocessed_fMRI_bold.nii.gz'))

S17


/Users/lukechang/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:20: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
/Users/lukechang/anaconda3/envs/py36/lib/python3.6/site-packages/nltools/stats.py:954: RuntimeWarning: invalid value encountered in sqrt
  stderr = np.sqrt(np.diag(np.linalg.pinv(np.dot(X.T, X))))[:, np.newaxis] * sigma[np.newaxis, :]
/Users/lukechang/anaconda3/envs/py36/lib/python3.6/site-packages/nltools/stats.py:963: RuntimeWarning: divide by zero encountered in true_divide
  t = b / stderr
/Users/lukechang/anaconda3/envs/py36/lib/python3.6/site-packages/nltools/stats.py:963: RuntimeWarning: invalid value encountered in true_divide
  t = b / stderr
/Users/lukechang/anaconda3/envs/py36/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:877: Runti

S10


/Users/lukechang/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:20: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
/Users/lukechang/anaconda3/envs/py36/lib/python3.6/site-packages/nltools/stats.py:954: RuntimeWarning: invalid value encountered in sqrt
  stderr = np.sqrt(np.diag(np.linalg.pinv(np.dot(X.T, X))))[:, np.newaxis] * sigma[np.newaxis, :]
/Users/lukechang/anaconda3/envs/py36/lib/python3.6/site-packages/nltools/stats.py:963: RuntimeWarning: divide by zero encountered in true_divide
  t = b / stderr
/Users/lukechang/anaconda3/envs/py36/lib/python3.6/site-packages/nltools/stats.py:963: RuntimeWarning: invalid value encountered in true_divide
  t = b / stderr
/Users/lukechang/anaconda3/envs/py36/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:877: Runti

S14


...

/Users/lukechang/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:20: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
/Users/lukechang/anaconda3/envs/py36/lib/python3.6/site-packages/nltools/stats.py:954: RuntimeWarning: invalid value encountered in sqrt
  stderr = np.sqrt(np.diag(np.linalg.pinv(np.dot(X.T, X))))[:, np.newaxis] * sigma[np.newaxis, :]
/Users/lukechang/anaconda3/envs/py36/lib/python3.6/site-packages/nltools/stats.py:963: RuntimeWarning: divide by zero encountered in true_divide
  t = b / stderr
/Users/lukechang/anaconda3/envs/py36/lib/python3.6/site-packages/nltools/stats.py:963: RuntimeWarning: invalid value encountered in true_divide
  t = b / stderr
/Users/lukechang/anaconda3/envs/py36/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:877: Runti

S04


/Users/lukechang/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:20: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
/Users/lukechang/anaconda3/envs/py36/lib/python3.6/site-packages/nltools/stats.py:954: RuntimeWarning: invalid value encountered in sqrt
  stderr = np.sqrt(np.diag(np.linalg.pinv(np.dot(X.T, X))))[:, np.newaxis] * sigma[np.newaxis, :]
/Users/lukechang/anaconda3/envs/py36/lib/python3.6/site-packages/nltools/stats.py:963: RuntimeWarning: divide by zero encountered in true_divide
  t = b / stderr
/Users/lukechang/anaconda3/envs/py36/lib/python3.6/site-packages/nltools/stats.py:963: RuntimeWarning: invalid value encountered in true_divide
  t = b / stderr
/Users/lukechang/anaconda3/envs/py36/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:877: Runti

S21


/Users/lukechang/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:20: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
/Users/lukechang/anaconda3/envs/py36/lib/python3.6/site-packages/nltools/stats.py:954: RuntimeWarning: invalid value encountered in sqrt
  stderr = np.sqrt(np.diag(np.linalg.pinv(np.dot(X.T, X))))[:, np.newaxis] * sigma[np.newaxis, :]
/Users/lukechang/anaconda3/envs/py36/lib/python3.6/site-packages/nltools/stats.py:963: RuntimeWarning: divide by zero encountered in true_divide
  t = b / stderr
/Users/lukechang/anaconda3/envs/py36/lib/python3.6/site-packages/nltools/stats.py:963: RuntimeWarning: invalid value encountered in true_divide
  t = b / stderr
/Users/lukechang/anaconda3/envs/py36/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:877: Runti

# Split Betas into Separate Files

In [17]:
df = pd.read_csv(os.path.join(data_dir, 'Design_Matrix.csv'))
file_list = glob(os.path.join(data_dir, '*', 'beta_denoised_smoothed_preprocessed_fMRI_bold.nii.gz'))

for f in file_list:
    sub = os.path.dirname(f).split('/')[-1]
    dat = Brain_Data(f)
    for i in range(df.shape[1]):
        dat[i].write(os.path.join(data_dir, sub, f"{sub}_beta_{df.columns[i]}.nii.gz"))